In [0]:
# source : https://www.kaggle.com/nafisur/keras-models-lstm-cnn-gru-bidirectional-glove
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
gc.collect()

0

In [0]:
train = pd.read_csv('./drive/My Drive/data/moives/train.tsv',sep='\t')
print(train.shape)
train.head()

(156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
test=pd.read_csv('./drive/My Drive/data/moives/test.tsv',sep='\t')
print(test.shape)
test.head()

(66292, 3)


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [0]:
sub=pd.read_csv('./drive/My Drive/data/moives/sampleSubmission.csv')
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [0]:
test['Sentiment']=-999
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine effort .,-999
1,156062,8545,An intermittently pleasing but mostly routine effort,-999
2,156063,8545,An,-999
3,156064,8545,intermittently pleasing but mostly routine effort,-999
4,156065,8545,intermittently pleasing but mostly routine,-999


In [0]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

(222352, 4)


,PhraseId,SentenceId,Phrase,Sentiment
222347,222348,11855,"A long-winded , predictable scenario .",-999
222348,222349,11855,"A long-winded , predictable scenario",-999
222349,222350,11855,"A long-winded ,",-999
222350,222351,11855,A long-winded,-999
222351,222352,11855,predictable scenario,-999


In [0]:
del train,test
gc.collect()

57

In [0]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [0]:
#数据清洗,大写转小写,删除标点符号
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [0]:
df['clean_review']=clean_review(df.Phrase.values)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [0]:
df_train=df[df.Sentiment!=-999]
df_train.shape

(156060, 5)

In [0]:
df_test=df[df.Sentiment==-999]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_test.shape)
df_test.head()

(66292, 4)


,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,An intermittently pleasing but mostly routine effort .,an intermittently pleasing but mostly routine effort
156061,156062,8545,An intermittently pleasing but mostly routine effort,an intermittently pleasing but mostly routine effort
156062,156063,8545,An,an
156063,156064,8545,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156064,156065,8545,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


In [0]:
del df
gc.collect()

23

In [0]:
train_text=df_train.clean_review.values
test_text=df_test.clean_review.values
target=df_train.Sentiment.values
#分类数据转one hot
y=to_categorical(target)

print(train_text.shape,target.shape,y.shape)

(156060,) (156060,) (156060, 5)


In [0]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(124848,) (124848, 5)
(31212,) (31212, 5)


In [0]:
#Finding number of unique words in train set

all_words=' '.join(X_train_text) # 将array中的所有字符串连接成一个字符串
all_words=word_tokenize(all_words)#对单词进行分割
dist=FreqDist(all_words) #计算词频wordcount
num_unique_word=len(dist)
num_unique_word

13732

In [0]:
#Finding max length of a review in train set
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

48

##Building Keras LSTM model##

In [0]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=5

In [0]:
#Tokenize Text
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

In [0]:
#sequence padding
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

(124848, 48) (31212, 48) (66292, 48)


##1.LSTM model##

In [0]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1373200   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 64)          42240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 1,428,021
Trainable params: 1,428,021
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 351s 3ms/step - loss: 1.0755 - acc: 0.5782 - val_loss: 0.8735 - val_acc: 0.6457
Epoch 2/3
124848/124848 [==============================] - 349s 3ms/step - loss: 0.8335 - acc: 0.6573 - val_loss: 0.8306 - val_acc: 0.6621
Epoch 3/3
124848/124848 [==============================] - 348s 3ms/step - loss: 0.7727 - acc: 0.6808 - val_loss: 0.8180 - val_acc: 0.6643
CPU times: user 22min 11s, sys: 2min 23s, total: 24min 35s
Wall time: 17min 30s


In [0]:
y_pred1=model1.predict_classes(X_test,verbose=1)

66292/66292 [==============================] - 212s 3ms/step


In [0]:
sub.Sentiment=y_pred1
sub.to_csv('./drive/My Drive/data/moives/sub1_LSTM.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,2


##2. CNN##

In [0]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))

model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total para

In [0]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 13s 102us/step - loss: 0.9970 - acc: 0.6023 - val_loss: 0.8480 - val_acc: 0.6527
Epoch 2/3
124848/124848 [==============================] - 11s 90us/step - loss: 0.7847 - acc: 0.6772 - val_loss: 0.8190 - val_acc: 0.6626
Epoch 3/3
124848/124848 [==============================] - 11s 90us/step - loss: 0.7047 - acc: 0.7061 - val_loss: 0.8051 - val_acc: 0.6644
CPU times: user 37.2 s, sys: 6.94 s, total: 44.1 s
Wall time: 35.6 s


In [0]:
y_pred2=model2.predict_classes(X_test, verbose=1)

66292/66292 [==============================] - 3s 52us/step


In [0]:
sub.Sentiment=y_pred2
sub.to_csv('./drive/My Drive/data/moives/sub2_CNN.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


##3. CNN +GRU##

In [0]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(5,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 64)            19264     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 64)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 24, 128)           74112     
_________________________________________________________________
dropout_4 (Dropout)          (None, 24, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
__________

In [0]:
%%time
history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 66s 529us/step - loss: 1.0265 - acc: 0.5910 - val_loss: 0.8593 - val_acc: 0.6499
Epoch 2/3
124848/124848 [==============================] - 65s 523us/step - loss: 0.8064 - acc: 0.6694 - val_loss: 0.8172 - val_acc: 0.6606
Epoch 3/3
124848/124848 [==============================] - 64s 516us/step - loss: 0.7266 - acc: 0.6968 - val_loss: 0.8091 - val_acc: 0.6682
CPU times: user 3min 32s, sys: 23 s, total: 3min 55s
Wall time: 3min 16s


In [0]:
y_pred3=model3.predict_classes(X_test, verbose=1)

66292/66292 [==============================] - 32s 477us/step


In [0]:
sub.Sentiment=y_pred3
sub.to_csv('./drive/My Drive/data/moives/sub3_CNN_GRU.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


##4. Bidirectional GRU##

In [0]:
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(5, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               175872    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 1285      
Total params: 1,550,357
Trainable params: 1,550,357
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 202s 2ms/step - loss: 1.0023 - acc: 0.5960 - val_loss: 0.8527 - val_acc: 0.6498
Epoch 2/3
124848/124848 [==============================] - 200s 2ms/step - loss: 0.8033 - acc: 0.6691 - val_loss: 0.8081 - val_acc: 0.6715
Epoch 3/3
124848/124848 [==============================] - 201s 2ms/step - loss: 0.7342 - acc: 0.6957 - val_loss: 0.8075 - val_acc: 0.6731
CPU times: user 11min 51s, sys: 1min 19s, total: 13min 10s
Wall time: 10min 4s


In [0]:
y_pred4=model4.predict_classes(X_test, verbose=1)

66292/66292 [==============================] - 107s 2ms/step


In [0]:
sub.Sentiment=y_pred4
sub.to_csv('./drive/My Drive/data/moives/sub4_Bidirectional_GRU.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


##5. Glove word embedding##

In [0]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

In [0]:
# embedding matrix
EMBEDDING_FILE = './drive/My Drive/data/glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

Found 400000 word vectors.
(13732, 100)


In [0]:
model5 = Sequential()
model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model5.add(SpatialDropout1D(0.25))
model5.add(Bidirectional(GRU(128,return_sequences=True)))
model5.add(Bidirectional(GRU(64,return_sequences=False)))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 48, 100)           1373200   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 48, 100)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 48, 256)           175872    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               123264    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 645       
Total params: 1,672,981
Trainable params: 1,672,981
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time
history5=model5.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=4, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/4
124848/124848 [==============================] - 389s 3ms/step - loss: 0.9966 - acc: 0.5914 - val_loss: 0.8396 - val_acc: 0.6542
Epoch 2/4
124848/124848 [==============================] - 387s 3ms/step - loss: 0.8483 - acc: 0.6497 - val_loss: 0.7916 - val_acc: 0.6759
Epoch 3/4
124848/124848 [==============================] - 385s 3ms/step - loss: 0.7870 - acc: 0.6724 - val_loss: 0.7744 - val_acc: 0.6793
Epoch 4/4
124848/124848 [==============================] - 384s 3ms/step - loss: 0.7450 - acc: 0.6921 - val_loss: 0.7648 - val_acc: 0.6822
CPU times: user 30min 1s, sys: 3min 6s, total: 33min 7s
Wall time: 25min 49s


In [0]:
y_pred5=model5.predict_classes(X_test, verbose=1)

66292/66292 [==============================] - 203s 3ms/step


In [0]:
sub.Sentiment=y_pred5
sub.to_csv('./drive/My Drive/data/moives/sub5_glove.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


##combine all output##

In [0]:
sub_all=pd.DataFrame({'model1':y_pred1,'model2':y_pred2,'model3':y_pred3,'model4':y_pred4,'model5':y_pred5})
pred_mode=sub_all.agg('mode',axis=1)[0].values
sub_all.head()

In [0]:
pred_mode=[int(i) for i in pred_mode]

In [0]:
sub.Sentiment=pred_mode
sub.to_csv('./drive/My Drive/data/moives/sub_mode.csv',index=False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
